In [1]:
# Data Science Libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import scipy.stats as stats

# Group-Created Zillow Functions
import wrangle

# Block Warning Boxes
import warnings
warnings.filterwarnings("ignore")


<hr style="border:2px solid blue"> </hr>


# Explore

In [2]:
df = wrangle.get_zillow_data()
df.head()

,bedrooms,bathrooms,squarefeet,year_built,regionidzip,fips,logerror
0,4.0,3.5,3100.0,1998.0,96978.0,6059.0,1023282.0
1,2.0,1.0,1465.0,1967.0,97099.0,6111.0,464000.0
2,3.0,2.0,1243.0,1962.0,97078.0,6059.0,564778.0
3,4.0,3.0,2376.0,1970.0,96330.0,6037.0,145143.0
4,4.0,3.0,2962.0,1950.0,96293.0,6037.0,773303.0


In [3]:
# Using the functions from my compiled wrangle file
train, validate, test = wrangle.wrangle_zillow()
train, validate, test = wrangle.Min_Max_Scaler(train, validate, test)

KeyError: 'tax_value'

In [ ]:
train.head()

In [ ]:
# Taking a look at correlations with my target variable
train.corr()['tax_value'].sort_values()

It looks like my top 3 are 
1. squarefeet
2. bathrooms
3. year_built / years_old

Considering that correlations are more for continuous variables, I can really only accept squarefeet from this list at the moment. You could almost consider bathrooms and year_built/years_old as categorical values.

In [ ]:
sns.pairplot(train, corner=True);

In [ ]:
# define function, arguments are dataframe and two lists of columns.
def plot_categorical_and_continuous_vars(df, continuous, categorical):
    '''
    Takes in a dataframe, a list of continuous variables, and a list of categorical 
    variables of the dataframe as arguments. Creates three different plots of each
    categorical pair.
    '''
    # loop through each column in the first list
    for con in continuous:
        # loop through each column in the second list, creating a loop for each categorical pair
        for cat in categorical:
            # each loop returns three subplots in a figure. Plots a swarmplot, stripplot, and boxplot.
            plt.figure(figsize = (20,10))
            plt.subplot(1,3,1)
            sns.swarmplot(x=df[cat], y=df[con], data=df)
            plt.subplot(1,3,2)
            sns.stripplot(x=df[cat], y=df[con], data=df)
            plt.subplot(1,3,3)
            sns.boxplot(x=df[cat], y=df[con], data=df)

In [ ]:
continuous = ['tax_value','squarefeet']
categorical= ['bedrooms','bathrooms','years_old','regionidzip']
train.sample(10_000)

In [ ]:
# DOn't run this until finished
#plot_categorical_and_continuous_vars(train.sample(10_000),continuous,categorical )

### Let's do some statistical questions.  
#### 1. Are homes with more bedrooms worth more?

In [ ]:
# Setting my confidence level to 95%
alpha = 0.05

In [ ]:
# I'm going to create some array's for this question
tax_value_2Bd = train[train.bedrooms == 2].tax_value
tax_value_5Bd = train[train.bedrooms == 5].tax_value

# Check to see if my variances are equal; (they are not)
# tax_value_2Bd.var() , tax_value_5Bd.var()

# Define my hypothesis before testing:
# H0: There is no difference in tax_value between homes with 2 bedrooms vs homes with 5 bedrooms.
# Ha: There is a difference in tax_value between homes with 2 bedrooms vs homes with 5 bedrooms.

# This is a two-sampled, two-tailed test. 
t, p = stats.ttest_ind(tax_value_2Bd, tax_value_5Bd, equal_var=False)

if p < alpha:
    print("We reject the null hypothesis")

else:
    print("We fail to reject the null hypothesis")

It didn't seem like there was much difference between home value bedrooms in the charts above, but my Two-Tailed T-Test proved otherwise. It seems more bedrooms does mean more value, which seems intuitive, but it's always good to check and see if your data agrees with you.

#### 2. Are younger homes higher in tax_value than old homes?

In [ ]:
# Defining the arrays i want for my t-test
older_homes = train[train.years_old >= train.years_old.mean()].tax_value
younger_homes = train[train.years_old < train.years_old.mean()].tax_value

In [ ]:
# Define my hypothesis before testing:
# H0: Homes older than the average cost more or the same as homes younger than the average
# Ha: Homes older than the average cost less than homes younger than the average

# use stats.ttest to calculate t and p
t, p = stats.ttest_ind(older_homes, younger_homes, equal_var=False)

if p/2 < alpha:
    print("We reject the null hypothesis")

else:
    print("We fail to reject the null hypothesis")

As we expected, the data shows that new homes have higher tax_value than older homes.

#### 3. Are homes with more bathrooms worth more?

In [ ]:
train.bathrooms.value_counts()

In [ ]:
# I'm going to create some array's for this question
tax_value_1Bth = train[train.bathrooms == 1].tax_value
tax_value_4Bth = train[train.bathrooms == 4].tax_value

# Check to see if my variances are equal; (they are not)
# tax_value_2Bd.var() , tax_value_5Bd.var()

# Define my hypothesis before testing:
# H0: There is no difference in tax_value between homes with 2 bedrooms vs homes with 5 bedrooms.
# Ha: There is a difference in tax_value between homes with 2 bedrooms vs homes with 5 bedrooms.

# This is a two-sampled, two-tailed test. 
t, p = stats.ttest_ind(tax_value_2Bd, tax_value_5Bd, equal_var=False)

if p < alpha:
    print("We reject the null hypothesis")

else:
    print("We fail to reject the null hypothesis")

1. Why do some properties have a much higher value than others when they are located so close to each other?
2. Are houses with a high bedroom count but a low bathroom count, less valueable?
3. Are houses of a certain year built more valuable than others?
4. Are there certain countys that are harder to predict taxvaluedollarcnt for?

I wanna see what my most valuable features are according to these feature selection models. To get started, I'll need to split my data and use the train dataset.

In [ ]:
# Wrangling my data
# Then assigning my features to X, and target variable to y, for train dataframe
train, validate, test = wrangle.wrangle_zillow()
X_train = train.drop(columns=['tax_value'])
y_train = train.tax_value


# Select Kbest 

In [ ]:
from sklearn.feature_selection import SelectKBest, f_regression

# Parameters: f_regression stats test, give me 2 features
f_selector = SelectKBest(f_regression, k=2)

# Find the top 2 X's correlated with y
f_selector.fit(X_train, y_train)

# Boolean mask of whether the column was selected or not. 
feature_mask = f_selector.get_support()

# Get list of top K features. 
f_feature = X_train.iloc[:,feature_mask].columns.tolist()

print("KBest Top Two Features:", f_feature)

# Recursive Feature Elimination

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE

# initialize the ML algorithm
lm = LinearRegression()

# create the rfe object, indicating the ML object (lm) and the number of features I want to end up with. 
rfe = RFE(lm, 2)

# fit the data using RFE
rfe.fit(X_train,y_train)  

# get the mask of the columns selected
feature_mask = rfe.support_

# get list of the column names. 
rfe_feature = X_train.iloc[:,feature_mask].columns.tolist()

print("Recursive Feature Elimination Top Two Features:",rfe_feature)

In [ ]:
# view list of columns and their ranking

# get the ranks
var_ranks = rfe.ranking_
# get the variable names
var_names = X_train.columns.tolist()
# combine ranks and names into a df for clean viewing
rfe_ranks_df = pd.DataFrame({'Var': var_names, 'Rank': var_ranks})
# sort the df by rank
rfe_ranks_df.sort_values('Rank')

Alright, so two different models put bathrooms at the top of the list (Kbest comes in reverse order). With this, I will assume my top three features to be Bathrooms, Bedrooms, and Squarefeet. I'll explore these further with some statistical tests. 

In [ ]:
col_list = ['bedrooms', 'bathrooms', 'total_living_area']
for col in col_list:
    sns.lmplot(x=col, y="tax_value", data=train, line_kws={'color': 'red'})
    plt.show()

<span style="color: red">

  
# Explore (.ipynb) - Ask clear question, [discover], provide a clear answer
At least 4 of the questions asked and answered of the data are shared in the final report notebook. You should call out questions of the data using natural language that speaks to the business stakeholders in markdown cells, ideally a header prior to the visualization or statistical test, that you then explore. This does not take the place of stating your null hypothesis/alternative hypothesis when doing a statistical test. But those hypotheses are generally for you. By writing questions that you intend to answer with visualizations and statistical tests in natural language, like "Are office supplies leading to differences in profit in Texas?", you are able to guide both yourself and your reader through the highlights of your analysis. You ask a question, create a visual, run a statistical test (if appropriate), and wrap it nicely with a markdown cell that contains a clear answer in layman's terms. You do all that before moving to the next question.

# Explore (.ipynb) - Exploring though visualizations
At least 5 visualizations are included in your final report. The ones included answer a question (remember, NO is an answer) and the one(s) to provide necessary context (such as the distribution of the target variable). All statistical tests included in the final report should be supported with an visualization of the interaction of the variables being tested. Charts in the final report should have titles and labels that are descriptive and useful for the end user/audience/consumer of the report. All visualizations in the final report are mentioned or discussed if a verbal presentation is given.

# Explore (.ipynb) - Statistical tests
At least 2 statistical tests are included in your final report. The correct tests are run, given the data type and distribution, and the correct conclusions are drawn. - correlation: 2 continuous variables, normally distributed, testing for LINEAR correlation only (H_0: Not linearly dependent) - independent t-test: 1 continuous, somewhat normally distributed variable, one boolean variable, equal variance, independent (H_0: population mean of each group is equal) - chi-square test: 2 discrete variables. (H_0: the 2 variables are independent of each other). (other tests may be used)
    
# Explore (.ipynb) - Summary
Following your exploration section, you summarize your analysis (in a markdown cell using natural language): what you found and how you will use it moving forward. This includes key takeaways from all the questions answered in explore, a list of which features will be used in modeling and why, and which features will not move forward and why. You may only call out a few of these features in the presentation, but having that there for reference is important in a report. A group of features may have the same reason why, and those can be mentioned together.

</span>